In [1]:
import numpy as np
import pandas as pd
import itertools as it

In [152]:
class Die():
    '''
    Docstring
    '''
    def __init__(self, instance_name, face_symbols):
        # Name of instance for future reference
        self.name = instance_name
        
        # Test for numpy array
        if not isinstance(face_symbols, np.ndarray):
            raise TypeError('Face Argument must be NumPy array')
          
        # Tests that array is strings or numbers
        if face_symbols.dtype.char not in ['U', 'l', 'd']:
            raise TypeError('Faces must be strings or numbers')
        
        # Tests for unique faces
        if len(set(face_symbols)) != len(face_symbols):
            raise ValueError('Face values must be distinct')
        
        # Initialize weights as 1.0 for each face
        self.faces = face_symbols
        self.n_sides = len(face_symbols)
        self.weights = [1.0 for i in face_symbols]
        
        # Save faces and weights in private data frame
        self._faces_weights = pd.DataFrame(self.weights, self.faces, columns=['weight'])
        
    def change_side_weight(self, face, new_weight):
        if face not in self._faces_weights.index:
            raise IndexError('That face does not exist on this die')
        if type(new_weight) not in [int, float]:
            raise ValueError('Weight is not a valid type')
        
        self._faces_weights.loc[face] = new_weight
        
    def roll(self, n = 1):
        results = []
        self.probs = [i/sum(self._faces_weights.weight) for i in self._faces_weights.weight]
        
        for i in range(n):
            result = self._faces_weights.sample(weights = self.probs).index.values[0]
            results.append(result)
            
        return results
        
    def current_state(self):
        print(self._faces_weights)
        
class Game():
    
    def __init__(self, die_list):
        self.die_list = die_list
    
    def play(self, n):
        self.df_index = []
        for i in range(n):
            self.df_index.append('Roll ' + str(i + 1))
        
        self.play_results = pd.DataFrame([], self.df_index)
        
        for die in self.die_list:
            self.play_results.insert(self.die_list.index(die), self.die_list.index(die) + 1, die.roll(n))
    
    def last_round(self, form = 'wide'):
        if form == 'narrow':
            return self.play_results.stack().to_frame('Value')
        elif form == 'wide':
            return self.play_results
        else:
            raise ValueError('Must request for a "narrow" or "wide" table')
        
class Analyzer():
    def __init__(self, game_object):
        if not isinstance(game_object, Game):
            raise ValueError('Passed value is not a Game object')
        
        self.game = game_object
        
        self.outcome = game_object.play_results
    
    def jackpot(self):
        count = 0
        for i in self.outcome.nunique(axis=1) == 1:
            if i == True:
                count += 1
        
        return count
    
    def face_count(self):
        return pd.DataFrame(self.outcome).apply(pd.Series.value_counts, axis = 1).fillna(0)
    
    def combo_count(self, sort = True):
        sorted_outcome = self.outcome.apply(sorted, axis = 1, result_type = 'broadcast')
        return sorted_outcome.value_counts(sort = sort).to_frame('Combination Count')
    
    def permutation_count(self, sort = True):
        return self.outcome.value_counts(sort = sort).to_frame('Permutation Count')

In [153]:
faces = np.array([1, 2, 3, 4, 5, 6])

In [154]:
die1 = Die('die1', faces)
die2 = Die('die2', faces)
die3 = Die('die3', faces)
die4 = Die('die4', faces)
die5 = Die('die5', faces)
die6 = Die('die6', faces)

In [155]:
die_list = [die1, die2]

In [156]:
jeff = Game(die_list)

In [157]:
jeff.play(20)

In [158]:
jeff.last_round()

,1,2
Roll 1,2,5
Roll 2,6,1
Roll 3,4,6
Roll 4,5,3
Roll 5,6,4
Roll 6,2,6
Roll 7,3,3
Roll 8,6,5
Roll 9,1,2
Roll 10,4,1


In [159]:
jeff_analyzer = Analyzer(jeff)

In [160]:
jeff_analyzer.jackpot()

1

In [161]:
jeff_analyzer.face_count()

,1,2,3,4,5,6
Roll 1,0.0,1.0,0.0,0.0,1.0,0.0
Roll 2,1.0,0.0,0.0,0.0,0.0,1.0
Roll 3,0.0,0.0,0.0,1.0,0.0,1.0
Roll 4,0.0,0.0,1.0,0.0,1.0,0.0
Roll 5,0.0,0.0,0.0,1.0,0.0,1.0
Roll 6,0.0,1.0,0.0,0.0,0.0,1.0
Roll 7,0.0,0.0,2.0,0.0,0.0,0.0
Roll 8,0.0,0.0,0.0,0.0,1.0,1.0
Roll 9,1.0,1.0,0.0,0.0,0.0,0.0
Roll 10,1.0,0.0,0.0,1.0,0.0,0.0


In [163]:
jeff_analyzer.permutation_count(False)

Permutation Count
1 2                   
1 2                  3
  4                  3
2 4                  1
  5                  1
  6                  1
3 3                  1
4 1                  2
  3                  1
  6                  1
5 3                  1
  4                  1
6 1                  1
  2                  1
  4                  1
  5                  1

In [165]:
jeff_analyzer.combo_count(False)

Combination Count
1 2                   
1 2                  3
  4                  5
  6                  1
2 4                  1
  5                  1
  6                  2
3 3                  1
  4                  1
  5                  1
4 5                  1
  6                  2
5 6                  1